In [3]:
# import numpy as np
# 
# # Load the distance matrix
# distance_matrix = np.load("identity_matrix_mmseq2.npy")

In [4]:
# distance_matrix[500:600, 500:600]

array([[100.   ,   0.698,   0.591, ...,   0.   ,   0.   ,   0.   ],
       [  0.71 , 100.   ,   0.64 , ...,   0.   ,   0.   ,   0.   ],
       [  0.605,   0.635, 100.   , ...,   0.   ,   0.   ,   0.   ],
       ...,
       [  0.   ,   0.   ,   0.   , ..., 100.   ,   0.583,   0.578],
       [  0.   ,   0.   ,   0.   , ...,   0.58 , 100.   ,   0.923],
       [  0.   ,   0.   ,   0.   , ...,   0.572,   0.92 , 100.   ]])

In [5]:
# distance_matrix = (distance_matrix + distance_matrix.T) / 2

In [6]:
# distance_matrix

array([[100.    ,   0.7155,   0.7145, ...,   0.    ,   0.    ,   0.    ],
       [  0.7155, 100.    ,   0.965 , ...,   0.    ,   0.    ,   0.    ],
       [  0.7145,   0.965 , 100.    , ...,   0.    ,   0.    ,   0.    ],
       ...,
       [  0.    ,   0.    ,   0.    , ..., 100.    ,   0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    , ...,   0.    , 100.    ,   0.    ],
       [  0.    ,   0.    ,   0.    , ...,   0.    ,   0.    , 100.    ]])

In [7]:
# with open('distance_matrix_mean.npy', 'wb') as f:
#     np.save(f, distance_matrix)


In [1]:
import numpy as np

distance_matrix = np.load("distance_matrix_mean.npy")

In [ ]:
# from sklearn.manifold import MDS
# 
# mds = MDS(n_components=2, dissimilarity="precomputed")
# feature_matrix = mds.fit_transform(distance_matrix)

/Users/salihtalhaakgun/miniconda3/envs/thesis/lib/python3.10/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [2]:
distance_matrix

array([[100.    ,   0.7155,   0.7145, ...,   0.    ,   0.    ,   0.    ],
       [  0.7155, 100.    ,   0.965 , ...,   0.    ,   0.    ,   0.    ],
       [  0.7145,   0.965 , 100.    , ...,   0.    ,   0.    ,   0.    ],
       ...,
       [  0.    ,   0.    ,   0.    , ..., 100.    ,   0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    , ...,   0.    , 100.    ,   0.    ],
       [  0.    ,   0.    ,   0.    , ...,   0.    ,   0.    , 100.    ]])

In [5]:
import pandas as pd

labels = pd.read_csv("./../labels_sf.csv")

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda_transformed = lda.fit_transform(feature_matrix, labels)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(lda_transformed[:, 0], lda_transformed[:, 1], c=labels, cmap='viridis')
plt.colorbar()
plt.title("LDA on Distance Matrix")
plt.show()

In [7]:
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform

# Suppose M is your distance matrix (n x n)
M = distance_matrix  # load or define your distance matrix
labels_sf = labels["SF"]

# SciPy linkage expects condensed distance matrix:
condensed_dist = squareform(M, checks=False)

# Perform hierarchical clustering (e.g., Ward's method - though strictly it expects Euclidean distances)
Z = linkage(condensed_dist, method='average')  # or 'complete', 'single', etc.

# Cut the dendrogram to get a specific number of clusters
n_clusters = 3
cluster_labels = fcluster(Z, n_clusters, criterion='maxclust')


In [8]:
cluster_labels

array([3, 3, 3, ..., 3, 3, 3], dtype=int32)

In [6]:
from sklearn.manifold import MDS

# Use MDS to get a 2D embedding from your distance matrix
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
X_2d = mds.fit_transform(distance_matrix)  # X_2d will be shape (n, 2)


/Users/salihtalhaakgun/miniconda3/envs/thesis/lib/python3.10/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


KeyboardInterrupt: 

In [10]:
def intrinsic_dim_2nn_from_dist_matrix(D):
    """
    Compute the local 2-Nearest-Neighbors (2NN) intrinsic dimensionality
    for each point, given a distance matrix D.
    
    Parameters
    ----------
    D : ndarray of shape (n_samples, n_samples)
        Distance matrix where D[i, j] is the distance from point i to point j.
        We expect D[i, i] = 0 (distance to itself).
    
    Returns
    -------
    local_id : ndarray of shape (n_samples,)
        The 2NN intrinsic dimension estimates for each sample.
    """
    n = D.shape[0]
    local_id = np.zeros(n, dtype=float)

    for i in range(n):
        # Extract distances from point i to others
        # Ignore the diagonal (distance to itself = 0), we don't consider that a neighbor.
        row = D[i].copy()
        row[i] = np.inf  # So it doesn't get chosen as a "neighbor"

        # Sort distances in ascending order
        sorted_dists = np.sort(row)
        
        # The 1st neighbor distance (r1) is the smallest non-diagonal distance
        r1 = sorted_dists[0]
        # The 2nd neighbor distance (r2) is the second-smallest
        r2 = sorted_dists[1]

        # Compute the 2NN local ID = ln(2) / ln(r2 / r1)
        # Watch out for numerical issues: 0 distances, identical distances, etc.
        if r1 <= 0 or r2 <= 0 or r2 == r1:
            local_id[i] = np.nan
        else:
            local_id[i] = np.log(2.0) / np.log(r2 / r1)
    
    return local_id


In [12]:
intrinsic_dim_2nn_from_dist_matrix((distance_matrix - 1) * -1)

array([197.54674378,   0.33079702,   0.33024402, ...,          nan,
                nan,          nan])